In [1]:
### Model ####
MODELNUM = 2
SCHEME = 'PrevailingWind_6Split'
FEATURES = ['Lat', 'Lon', 'Alt', 'Temp', 'Precip', 'Year', 'JulianDay_Sin']

### Import Libraries
# Base Libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import json
# Tensorflow, scikit, kerasTuner
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from sklearn.preprocessing import MinMaxScaler
import keras_tuner as kt

In [2]:
# Function to import a dataset and transform headers for easier coding and convert Date column
# Pseudocode:
# 1. Import Dataset Save old Headers for later
# 2. Transform headernames to rid of units
# 3. Convert Date into Year and Julian Day
# 4. Peform a Sine Transformation on JulianDay
# 5. Return the dataset, and old headers
def importData(fileName):
    # Read in the correct file
    dataset = pd.read_csv(f'../../Data/{fileName}.csv')
    oldCols = list(dataset.columns)

    # Remove any units (anything in parentheses)
    codeCols = list(map(lambda x: re.sub(r'\(([^()]*)\)', '', x).strip(), oldCols))
    dataset.columns = codeCols

    # Transform Date into Year and JulianDay_Sin
    dataset['Date'] = pd.to_datetime(dataset['Date'], utc=True)
    dataset['Year'] = dataset['Date'].dt.year
    dataset['JulianDay'] = dataset['Date'].dt.dayofyear
    # Sine transformation to account for cyclical nature of Julian Day
    dataset['JulianDay_Sin'] = np.sin(2*np.pi*dataset['JulianDay']/365) 
    dataset.drop(columns=['Date', 'JulianDay'], inplace=True)
    
    #Add year and JulianDay_Sin to oldCols 
    oldCols += ['Year', 'JulianDay_Sin']
    
    return dataset, oldCols

In [4]:
importData(f'DataTest')[0]

,Lat,Lon,Alt,Precip,Temp,O18,H2,KPN_A,KPN_B,KPN_C,KPN_D,KPN_E,Year,JulianDay_Sin
0,46.491434,9.898181,1724.0,4.920363,6.623712,-8.25,-57.00,0,0,0,0,1,1999,0.296713
1,22.316667,114.166667,66.0,455.500000,29.100000,-8.17,-52.10,0,0,1,0,0,2010,-0.213521
2,43.732389,7.423583,2.0,372.000000,19.000000,-6.50,-45.00,0,0,1,0,0,2000,-0.711657
3,45.320000,-75.670000,114.0,130.000000,17.800000,-8.20,-58.00,0,0,0,1,0,2017,0.296713
4,-0.841111,29.941944,1595.0,4.519369,24.926385,-1.69,3.20,1,0,0,0,0,2005,-0.958718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18834,48.828100,9.200000,314.0,29.900000,1.600000,-14.64,-112.35,0,0,1,0,0,2008,-0.271958
18835,0.453056,33.195000,1181.0,0.000000,21.978937,-3.80,-22.30,1,0,0,0,0,2003,-0.288482
18836,32.366667,-6.400000,468.0,4.500000,11.800000,-0.39,6.07,0,0,1,0,0,2003,0.085965
18837,41.270000,69.270000,428.0,40.000000,12.000000,-5.00,-26.00,0,0,1,0,0,1971,-0.723644
